## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *

from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import expon
from scipy.stats import binom

from brainpipe.statistics import *
from os import path
from mne.stats import *

from mne.baseline import rescale
from mne.filter import filter_data

## User variables

In [3]:
# where to find data
st = study('Olfacto')
score = 'Epi' #'Rec'
if score == 'Epi':
    path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_Good_Bad_EpiScore/')
    save_path = path.join(st.path, 'classified/0_Classif_ERP_EpiScore_all_points_all_electrodes/')
if score == 'Rec':
    path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_RecScore/')
    save_path = path.join(st.path, 'classified/0_Classif_ERP_RecScore_all_points_all_electrodes/')

# ANALYSIS PARAMETERS
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [973 , 1024] #100ms before odor perception
data_to_use = [973, 1536] #1000ms after odor
time_points = data_to_use[1]-data_to_use[0]
classif = ['lda']
n_rep = 10 #bootstrap
alpha = 0.05

-> Olfacto loaded


## ERPs Decoding - Good Bad Odors Encoding

In [ ]:
test = True

if test == True:
    n_elec = {'PIRJ' :1}
    subjects = ['PIRJ']
else :
    subjects = ['VACJ','SEMC','PIRJ','LEFC','MICP','CHAF'] 
    n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
        }
    
elif test == 'subset': #HC, PHC, Amg, Pir
    subjects = ['VACJ', 'SEMC','PIRJ','LEFC'] #MICP
    n_elec = {
    'VACJ' : [1,2,3,11,12,13,14,15,16,17,22,23,24,60,61,62],
    'SEMC' : [0,1,2,3,4],
    'PIRJ' : [0,1,2,3,4,11,12,13,14,15,16,22,23,24,25,26,33,34,35,36,37,38],
    'LEFC' : [0,1,11,12,13,14,22,23,24,25,26,27],
    'MICP' : [0,1,2,3,9,18,10,11,12,19,20,21,22,23,29,30,31,32,33,40,41,42,43,44],
        }

for su in subjects:
    #Load files
    data_bad = np.load(path.join(path_data, su+'_concat_odor_bad_bipo.npz'))
    data_good = np.load(path.join(path_data, su+'_concat_odor_good_bipo.npz'))
    data_bad, channels, names, data_good = data_bad['x'], data_bad['channel'], data_bad['label'], data_good['x']

    for elec in range(0,n_elec[su]):
    #for elec in n_elec[su]:
        # Select data for one elec + name :
        data_elec_bad = data_bad[elec,:,:]
        data_elec_good = data_good[elec,:,:]
        ntrials = str(data_elec_bad.shape[1])+'/'+ str(data_elec_good.shape[1])
        channel, name = channels[elec], names[elec]
        print (su, 'Channel : ', channel, 'Label : ', name,'Bad shape : ', 
               data_elec_bad.shape, 'Good shape : ', data_elec_good.shape)

        #Filter data for one elec (all trials):
        data_elec_bad = np.array(data_elec_bad, dtype='float64')
        data_elec_good = np.array(data_elec_good, dtype='float64')
        data_bad_to_filter = np.swapaxes(data_elec_bad, 0, 1)
        data_good_to_filter = np.swapaxes(data_elec_good, 0, 1)
        filtered_data_bad = filter_data(data_bad_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        filtered_data_good = filter_data(data_good_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        print ('Size of filtered data bad :', filtered_data_bad.shape, 'filtered data good : ', filtered_data_good.shape,)

        #Normalize the non-averaged data (all trials)
        times = np.arange(filtered_data_bad.shape[1])
        print ('time points : ', times.shape)
        norm_filtered_data_bad = rescale(filtered_data_bad, times=times, baseline=baseline, mode=norm_mode)
        norm_filtered_data_good = rescale(filtered_data_good, times=times, baseline=baseline, mode=norm_mode)
        print ('Size norm & filtered data 0 : ', norm_filtered_data_bad.shape, norm_filtered_data_good.shape,)

        # Range of the data to compute
        data_range = range(data_to_use[0], data_to_use[1])
        sel_bad = norm_filtered_data_bad[:, data_range]
        sel_good = norm_filtered_data_good[:, data_range,]
        print ('-> Shape of bad data', sel_bad.shape, 'good data', sel_good.shape)

        # Average the signal on consecutive windows
        winSample = 10 #20ms in samples
        n_pts = sel_bad.shape[1]
        rmPoints = n_pts % winSample # Points to remove before splitting
        shapeRmPoints = np.arange(n_pts-rmPoints).astype(int) # Number of points for round division
        n_win = int(n_pts / winSample) # Number of segments

        # Split and average data (trials, n_pts)
        bad_split = np.array(np.split(sel_bad[:, shapeRmPoints], n_win, axis=1)) # n_win n_trials n_pts
        bad_split = np.mean(bad_split, axis=2).swapaxes(0,1) # n-trials n_win
        good_split = np.array(np.split(sel_good[:, shapeRmPoints], n_win, axis=1))
        good_split = np.mean(good_split, axis=2).swapaxes(0,1)

# ================================  STATISTICS FOR POWER  =====================================
        T_rep, p_val_rep = np.array([]), np.array([])
        da_rep, daperm_rep = np.array([]), np.array([])

        first = True
        for i in range(n_rep):
            #reshape data to have the exact same nb of trials (mandatory for t-tests)
            if bad_split.shape[0] > good_split.shape[0]:
                bad_split_stat = bad_split[np.random.randint(bad_split.shape[0], size=good_split.shape[0]), :] #reshape bad_data to fit good_data shape
                good_split_stat = good_split
            elif bad_split.shape[0] < good_split.shape[0]:
                bad_split_stat = bad_split
                good_split_stat = good_split[np.random.randint(good_split.shape[0], size=bad_split.shape[0]), :]
            else:
                bad_split_stat = bad_split
                good_split_stat = good_split
            ntrials = bad_split.shape[0]
            X = bad_split_stat - good_split_stat #the last dimension needs to be time
            T0, p_values, H0 = permutation_t_test(X, n_permutations=100, tail=0, n_jobs=1, verbose=None)
            T_rep = np.vstack((T_rep,T0)) if np.size(T_rep) else T0
            p_val_rep = np.vstack((p_val_rep,p_values)) if np.size(p_val_rep) else p_values

# =============================  CLASSIFICATION COMPUTATION ============================================================           

            #create a data matrix, concatenate along the trial dimension
            bad_good = np.concatenate((bad_split_stat, good_split_stat), axis=0)
            print ('Size of the concatenated data: ', bad_good.shape, 'Number time windows : ', bad_good.shape[1])

            #create label vector (0 for rest and 1 for odor)
            y = [0]*bad_split_stat.shape[0] + [1]*good_split_stat.shape[0]
            print ('Size of label for classif: ', len(y))

            da_final = []
            for i in range(bad_good.shape[1]):
                if first:
                    cv = StratifiedKFold(n_splits=10)
                    clf = SVC(class_weight='balanced', kernel='rbf')
                    params = {'C': expon(scale=100), 'gamma': expon(scale=.1)}
                    RS = RandomizedSearchCV(estimator=clf,
                                param_distributions=params,
                                n_iter=100,
                                n_jobs=-1,
                                cv=cv)
                    RS.fit(X=bad_good[:,i].reshape(-1,1), y=y)
                    best_params = RS.best_params_
                    best_params['class_weight'] = 'balanced'
                    best_params['kernel'] = 'rbf'

                # Define a cross validation:
                cv = defCv(y, n_folds=10, cvtype='skfold', rep=10)
                # Define classifier technique
                clf = defClf(y=y, clf=classif, kern='rbf',
                             C=best_params['C'], gamma=best_params['gamma'],
                             class_weight=best_params['class_weight'])#,n_tree=200, random_state=100)
                #Classify rest and odor
                cl = classify(y, clf=clf, cvtype=cv)

                # Evaluate the classifier on data:
                da,pvalue,daperm = cl.fit(bad_good[:,i].reshape(-1,1), n_perm=100,method='bino',mf=False)
                da_final.append(da.tolist())
            da = np.asarray(da_final).squeeze().swapaxes(0,1)
            print(da.shape)
            print ('decoding accuracy',da.shape, 'pvalues ', pvalue.shape,)
            da_rep = np.vstack((da_rep,da)) if np.size(da_rep) else da
            first = False

# =============================== TAKE MAX STATS TO PLOT =====================================
        # Take the max pvalues for each time window
        idx_pval_max = []
        for s in range(n_win):
            pval_max = p_val_rep[:,s].max()
            idx_pval_max.append(pval_max)
        np.save(path2save+su+'_pval_stats_Bad_vs_Good_ERP_'+str(name)+'_('+str(elec)+')',idx_pval_max)

        #Save da accuracy
        np.save(path2save+su+'_da_Bad_vs_Good_ERP_'+classif+'_'+str(name)+'_('+str(elec)+')',da_rep)

# ============================== PLOT ERPs ANALYSIS + STATS & DECODING ACCURACY ===================================================
        # data to plot
        bad_good_plot = np.concatenate((bad_split, good_split), axis=0)
        y_plot = [0]*bad_split.shape[0] + [1]*good_split.shape[0]

        # plot and figure parameters
        xfmt = ScalarFormatter(useMathText=True)
        xfmt.set_powerlimits((0,3))
        fig = plt.figure(1,figsize=(7,7))
        title = 'ERP and DA for '+su+' Good/Bad '+str(channel)+' '+str(name)+' ('+str(elec)+') ntrials:'+str(ntrials)
        fig.suptitle(title, fontsize=12)
        times_plot = 1000 * np.arange((baseline[0] - baseline[1]), len(shapeRmPoints)-baseline[1]+baseline[0],winSample) / sf
        #print (times_plot)
        lines = [0] #time vector is in ms

        # Plot the ERPs
        plt.subplot(211)
        BorderPlot(times_plot, bad_good_plot, y=y_plot, kind='sem', alpha=0.2, color=['b','m'], 
                   linewidth=2, ncol=1, xlabel='Time (ms)',ylabel = r' $\mu$V', legend=['bad','good'])
        addLines(plt.gca(), vLines=lines, vColor=['r'], vWidth=[2], hLines=[0], 
                 hColor=['#000000'], hWidth=[2])
        addPval(plt.gca(), idx_pval_max, p=0.05, x=times_plot, y=5, color='0.5', lw=2)
        addPval(plt.gca(), idx_pval_max, p=0.01, x=times_plot, y=0.2, color='0.7', lw=2)
        addPval(plt.gca(), idx_pval_max, p=0.001, x=times_plot, y=0.3, color='0.9', lw=2)
        rmaxis(plt.gca(), ['right', 'top'])
        plt.legend(loc=0, handletextpad=0.1, frameon=False)
        plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

        # Plot DA for the ERPs
        plt.subplot(212)
        BorderPlot(times_plot, da_rep, color='b', kind='std',xlabel='Time (ms)', ylim=[da.min()-10,da.max()+10],
                   ylabel='Decoding accuracy (%)',linewidth=2,alpha=0.3)
        rmaxis(plt.gca(), ['right', 'top'])
        addLines(plt.gca(), vLines=lines, vWidth=[2], vColor=['r'], hLines=[50], 
                 hColor=['#000000'], hWidth=[2])
        plt.legend(loc=0, handletextpad=0.1, frameon=False)   
        plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
        th_0_05 = 100*np.around(binom.isf(0.05, bad_split.shape[0], 0.5)/bad_split.shape[0],2)
        th_0_01 = 100*np.around(binom.isf(0.01, bad_split.shape[0], 0.5)/bad_split.shape[0],2)
        th_0_001 = 100*np.around(binom.isf(0.001, bad_split.shape[0], 0.5)/bad_split.shape[0],2)
        plt.plot(times_plot, th_0_05*np.ones(len(times_plot)), '--', color='orange', 
                  linewidth=2, label= str(th_0_05)+' - p < .05')
        plt.plot(times_plot, th_0_01*np.ones(len(times_plot)), '--', color='orangered', 
                  linewidth=2, label= str(th_0_01)+' - p < .01')
        plt.plot(times_plot, th_0_001*np.ones(len(times_plot)), '--', color='r', 
                      linewidth=2, label= str(th_0_001)+' - p < .001')

# =========================== SAVE FIGURES & CLEAN MEMORY ==========================================================================
        #Save the plot
        fname = path.join(path2save, su+'_Bad_vs_Good_ERP_'+classif+'_'+str(name)+'_('+str(elec)+').png')
        fig.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()    
        del bad_split, good_split, good_split_stat, bad_split_stat
    del data_bad, data_good, channels, names

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'feature/8_PLV_EPISCORE_Good_Bad_across_time_subset/')

da= np.load(path_data+'LEFC_bad_plv_theta_elec_(0)_Amg-pPirT.npy')
print(da.shape)

In [ ]:
times_plot = 1000 * np.arange((baseline[0] - baseline[1]), len(shapeRmPoints)-baseline[1]+baseline[0],winSample) / sf
print (len(shapeRmPoints))
print (len(times_plot))

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_Good_Bad_EpiScore/')
path2save = path.join (st.path, 'classified/8_Classif_ERP_EPISCORE_Bad_Good_across_time_win20ms_subset/lda/')

test = 'subset'
classifs = ['lda']

if test == True:
    n_elec = {'VACJ' :50}
    subjects = ['VACJ']
    
elif test == False :
    subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ'] 
    n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
        }
    
elif test == 'subset': #HC, PHC, Amg, Pir
    subjects = ['MICP', 'VACJ', 'SEMC','PIRJ','LEFC'] #'MICP
    n_elec = {
    'VACJ' : [1,2,3,11,12,13,14,15,16,17,22,23,24,60,61,62],
    'SEMC' : [0,1,2,3,4],
    'PIRJ' : [0,1,2,3,4,11,12,13,14,15,16,22,23,24,25,26,33,34,35,36,37,38],
    'LEFC' : [0,1,11,12,13,14,22,23,24,25,26,27],
    'MICP' : [0,1,2,3,9,18,10,11,12,19,20,21,22,23,29,30,31,32,33,40,41,42,43,44],
        }

for classif in classifs:
    for su in subjects:
        #Load files
        data_bad = np.load(path.join(path_data, su+'_concat_odor_bad_bipo.npz'))
        data_good = np.load(path.join(path_data, su+'_concat_odor_good_bipo.npz'))
        data_bad, channels, names, data_good = data_bad['x'], data_bad['channel'], data_bad['label'], data_good['x']

        #for elec in range(0,n_elec[su]):
        for elec in n_elec[su]:
            # Select data for one elec + name :
            data_elec_bad = data_bad[elec,:,:]
            data_elec_good = data_good[elec,:,:]
            ntrials = str(data_elec_bad.shape[1])+'/'+ str(data_elec_good.shape[1])
            channel, name = channels[elec], names[elec]
            print (su, 'Channel : ', channel, 'Label : ', name,'Bad shape : ', 
                   data_elec_bad.shape, 'Good shape : ', data_elec_good.shape)

            #Filter data for one elec (all trials):
            data_elec_bad = np.array(data_elec_bad, dtype='float64')
            data_elec_good = np.array(data_elec_good, dtype='float64')
            data_bad_to_filter = np.swapaxes(data_elec_bad, 0, 1)
            data_good_to_filter = np.swapaxes(data_elec_good, 0, 1)
            filtered_data_bad = filter_data(data_bad_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
            filtered_data_good = filter_data(data_good_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
            print ('Size of filtered data bad :', filtered_data_bad.shape, 'filtered data good : ', filtered_data_good.shape,)

            #Normalize the non-averaged data (all trials)
            times = np.arange(filtered_data_bad.shape[1])
            print ('time points : ', times.shape)
            norm_filtered_data_bad = rescale(filtered_data_bad, times=times, baseline=baseline, mode=norm_mode)
            norm_filtered_data_good = rescale(filtered_data_good, times=times, baseline=baseline, mode=norm_mode)
            print ('Size norm & filtered data 0 : ', norm_filtered_data_bad.shape, norm_filtered_data_good.shape,)

            # Range of the data to compute
            data_range = range(data_to_use[0], data_to_use[1])
            sel_bad = norm_filtered_data_bad[:, data_range]
            sel_good = norm_filtered_data_good[:, data_range,]
            print ('-> Shape of bad data', sel_bad.shape, 'good data', sel_good.shape)

            # Average the signal on consecutive windows
            winSample = 10 #20ms in samples
            n_pts = sel_bad.shape[1]
            rmPoints = n_pts % winSample # Points to remove before splitting
            shapeRmPoints = np.arange(n_pts-rmPoints).astype(int) # Number of points for round division
            n_win = int(n_pts / winSample) # Number of segments

            # Split and average data (trials, n_pts)
            bad_split = np.array(np.split(sel_bad[:, shapeRmPoints], n_win, axis=1)) # n_win n_trials n_pts
            bad_split = np.mean(bad_split, axis=2).swapaxes(0,1) # n-trials n_win
            good_split = np.array(np.split(sel_good[:, shapeRmPoints], n_win, axis=1))
            good_split = np.mean(good_split, axis=2).swapaxes(0,1)

# ================================  STATISTICS FOR POWER  =====================================
            n_rep = 10
            T_rep, p_val_rep = np.array([]), np.array([])
            da_rep, daperm_rep = np.array([]), np.array([])
            alpha = 0.05
            
            for i in range(n_rep):
                #reshape data to have the exact same nb of trials (mandatory for t-tests)
                if bad_split.shape[0] > good_split.shape[0]:
                    bad_split_stat = bad_split[np.random.randint(bad_split.shape[0], size=good_split.shape[0]), :] #reshape bad_data to fit good_data shape
                    good_split_stat = good_split
                elif bad_split.shape[0] < good_split.shape[0]:
                    bad_split_stat = bad_split
                    good_split_stat = good_split[np.random.randint(good_split.shape[0], size=bad_split.shape[0]), :]
                else :
                    bad_split_stat = bad_split
                    good_split_stat = good_split                    
                ntrials = bad_split.shape[0]
                X = bad_split_stat - good_split_stat #the last dimension needs to be time
                T0, p_values, H0 = permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)
                T_rep = np.vstack((T_rep,T0)) if np.size(T_rep) else T0
                p_val_rep = np.vstack((p_val_rep,p_values)) if np.size(p_val_rep) else p_values

# =============================  CLASSIFICATION COMPUTATION ============================================================           
                                    
                #create a data matrix, concatenate along the trial dimension
                bad_good = np.concatenate((bad_split_stat, good_split_stat), axis=0)
                print ('Size of the concatenated data: ', bad_good.shape, 'Number time windows : ', bad_good.shape[1])

                #create label vector (0 for rest and 1 for odor)
                y = [0]*bad_split_stat.shape[0] + [1]*good_split_stat.shape[0]
                print ('Size of label for classif: ', len(y))

                # Define a cross validation:
                cv = defCv(y, n_folds=10, cvtype='skfold', rep=10)
                # Define classifier technique
                clf = defClf(y=y, clf=classif, kern='rbf')#,n_tree=200, random_state=100)
                #Classify rest and odor
                cl = classify(y, clf=clf, cvtype=cv)

                # Evaluate the classifier on data:
                da,pvalue,daperm = cl.fit(bad_good, n_perm=100,method='bino',mf=False)
                print(da.shape)
                print ('decoding accuracy',da.shape, 'pvalues ', pvalue.shape,)
            da_rep = np.vstack((da_rep,da)) if np.size(da_rep) else da
            print (da_rep.shape)
                
# =============================== TAKE MAX STATS TO PLOT =====================================
            # Take the max pvalues for each time window
            idx_pval_max = []
            for s in range(n_win):
                pval_max = p_val_rep[:,s].max()
                idx_pval_max.append(pval_max)
            np.save(path2save+su+'_pval_stats_Bad_vs_Good_ERP_'+str(name)+'_('+str(elec)+')',idx_pval_max)

            #Save da accuracy
            np.save(path2save+su+'_da_Bad_vs_Good_ERP_'+classif+'_'+str(name)+'_('+str(elec)+')',da_rep)

# ============================== PLOT ERPs ANALYSIS + STATS & DECODING ACCURACY ===================================================
            # data to plot
            bad_good_plot = np.concatenate((bad_split, good_split), axis=0)
            y_plot = [0]*bad_split.shape[0] + [1]*good_split.shape[0]

            # plot and figure parameters
            xfmt = ScalarFormatter(useMathText=True)
            xfmt.set_powerlimits((0,3))
            fig = plt.figure(1,figsize=(7,7))
            title = 'ERP and DA for '+su+' Good/Bad '+str(channel)+' '+str(name)+' ('+str(elec)+') ntrials:'+str(ntrials)
            fig.suptitle(title, fontsize=12)
            times_plot = 1000 * np.arange((baseline[0] - baseline[1]), len(shapeRmPoints)-baseline[1]+baseline[0],winSample) / sf
            #print (times_plot)
            lines = [0] #time vector is in ms

            # Plot the ERPs
            plt.subplot(211)
            BorderPlot(times_plot, bad_good_plot, y=y_plot, kind='sem', alpha=0.2, color=['b','m'], 
                       linewidth=2, ncol=1, xlabel='Time (ms)',ylabel = r' $\mu$V', legend=['bad','good'])
            addLines(plt.gca(), vLines=lines, vColor=['r'], vWidth=[2], hLines=[0], 
                     hColor=['#000000'], hWidth=[2])
            addPval(plt.gca(), idx_pval_max, p=0.05, x=times_plot, y=5, color='0.5', lw=2)
            addPval(plt.gca(), idx_pval_max, p=0.01, x=times_plot, y=0.2, color='0.7', lw=2)
            addPval(plt.gca(), idx_pval_max, p=0.001, x=times_plot, y=0.3, color='0.9', lw=2)
            rmaxis(plt.gca(), ['right', 'top'])
            plt.legend(loc=0, handletextpad=0.1, frameon=False)
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

            # Plot DA for the ERPs
            plt.subplot(212)
            BorderPlot(times_plot, da_rep, color='b', kind='std',xlabel='Time (ms)', ylim=[da.min()-10,da.max()+10],
                       ylabel='Decoding accuracy (%)',linewidth=2,alpha=0.3)
            rmaxis(plt.gca(), ['right', 'top'])
            addLines(plt.gca(), vLines=lines, vWidth=[2], vColor=['r'], hLines=[50], 
                     hColor=['#000000'], hWidth=[2])
            plt.legend(loc=0, handletextpad=0.1, frameon=False)   
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            th_0_05 = 100*np.around(binom.isf(0.05, bad_split.shape[0], 0.5)/bad_split.shape[0],2)
            th_0_01 = 100*np.around(binom.isf(0.01, bad_split.shape[0], 0.5)/bad_split.shape[0],2)
            th_0_001 = 100*np.around(binom.isf(0.001, bad_split.shape[0], 0.5)/bad_split.shape[0],2)
            plt.plot(times_plot, th_0_05*np.ones(len(times_plot)), '--', color='orange', 
                      linewidth=2, label= str(th_0_05)+' - p < .05')
            plt.plot(times_plot, th_0_01*np.ones(len(times_plot)), '--', color='orangered', 
                      linewidth=2, label= str(th_0_01)+' - p < .01')
            plt.plot(times_plot, th_0_001*np.ones(len(times_plot)), '--', color='r', 
                          linewidth=2, label= str(th_0_001)+' - p < .001')

# =========================== SAVE FIGURES & CLEAN MEMORY ==========================================================================
            #Save the plot
            fname = path.join(path2save, su+'_Bad_vs_Good_ERP_'+classif+'_'+str(name)+'_('+str(elec)+').png')
            fig.savefig(fname, dpi=300, bbox_inches='tight')
            plt.clf()
            plt.close()    
            del bad_split, good_split, good_split_stat, bad_split_stat
        del data_bad, data_good, channels, names